# **Reto II**

### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.

### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.

### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.

### Inicialización de SparkSession:

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 1")
                        .master("local")
                        .getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4f7b1723


In [3]:
val udfs = spark.read.format("csv")
                     .option("header", "true")
                     .option("delimiter", ";")
                     .load("./reto2/udfs.csv")

udfs: org.apache.spark.sql.DataFrame = [nb: string, contract: string ... 12 more fields]


In [23]:
udfs.printSchema()

root
 |-- nb: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- udf_ref: string (nullable = true)
 |-- fmly: string (nullable = true)
 |-- grp: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country: string (nullable = true)
 |-- udf_name: string (nullable = true)
 |-- num_value: string (nullable = true)
 |-- string_value: string (nullable = true)
 |-- date_value: string (nullable = true)
 |-- data_timestamp_part: string (nullable = true)
 |-- data_date_part: string (nullable = true)
 |-- source_system: string (nullable = true)



In [139]:
val aux=udfs
.withColumn("strings", when($"string_value"==="NULL" or $"string_value"==="null",null).otherwise($"string_value"))
.drop("string_value")
.withColumn("nums", when($"num_value"==="NULL" or $"num_value"==="null",null).otherwise($"num_value"))
.drop("num_value")
.groupBy("nb")
.pivot("udf_name")
.agg(first(coalesce($"nums",$"strings"))).drop("M_PRUEBA")

aux.show(50)

+---------+-----+--------+----------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
|       nb|M_CCY|M_CLIENT|      M_CRDTCHRG|      M_DIRECTIAV|   M_DISCMARGIN|   M_LIQDTYCHRG|         M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+---------+-----+--------+----------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
| 20513130| null|    CCMO|  0.000000000000|   0.000000000000|             10| 0.000000000000|            20|0.000000000000|      WATT|      5493|
| 18710605|  MXN|    null|            null|   0.000000000000|           null|           null|          null|          null|      AMAM|      null|
| 20533916| null|    CCMO|  0.000000000000|   0.000000000000|             50| 0.000000000000|            30|0.000000000000|      WATT|      1999|
| 23097010|  EUR|    null|            null|   0.000000000000|           null|           null|          null|          null| 

aux: org.apache.spark.sql.DataFrame = [nb: string, M_CCY: string ... 9 more fields]


In [140]:
val prueba=aux.filter(($"M_DISCMARGIN".isNotNull && ($"M_DISCMARGIN"!==0)) ||
                      ($"M_DIRECTIAV".isNotNull && ($"M_DIRECTIAV"!==0)) ||
                      ($"M_LIQDTYCHRG".isNotNull && ($"M_LIQDTYCHRG"!==0)) ||
                      ($"M_CRDTCHRG".isNotNull && ($"M_CRDTCHRG"!==0)) ||
                      ($"M_MVA".isNotNull && ($"M_MVA"!==0)) ||
                      ($"M_RVA".isNotNull && ($"M_RVA"!==0)) ||
                      ($"M_SELLER".isNotNull)) 

prueba.show(65)

+-----------+-----+--------+----------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
|         nb|M_CCY|M_CLIENT|      M_CRDTCHRG|      M_DIRECTIAV|   M_DISCMARGIN|   M_LIQDTYCHRG|         M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+-----------+-----+--------+----------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
|   20513130| null|    CCMO|  0.000000000000|   0.000000000000|             10| 0.000000000000|            20|0.000000000000|      WATT|      5493|
|   18710605|  MXN|    null|            null|   0.000000000000|           null|           null|          null|          null|      AMAM|      null|
|   20533916| null|    CCMO|  0.000000000000|   0.000000000000|             50| 0.000000000000|            30|0.000000000000|      WATT|      1999|
|   23097010|  EUR|    null|            null|   0.000000000000|           null|           null|          null|  

prueba: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [141]:
prueba.count()

res130: Long = 60


### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [142]:
val resultado = prueba

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [143]:
assert(resultado.count() == 60)
assert(resultado.columns.size == 11)
assert(resultado.columns(4) == "M_DIRECTIAV")
assert(resultado.select("M_SELLER").filter($"nb" === 23037162).first.getString(0) == "AMAM")
assert(resultado.select("M_SELLER").filter($"nb" === 19665186).first.getString(0) == "LB_VSTAVRE")
assert(resultado.select("M_RVA").filter($"nb" === 444111222).first.getString(0) == "8956")